In [20]:
import pandas as pd
import sqlite3

In [24]:
conn = sqlite3.connect('dialogue_analysis.db')

In [22]:
episodes_df = pd.read_csv('Sample Data/episodes.csv')
organizations_monthly_df = pd.read_csv('Sample Data/organizations_monthly.csv')
cost_to_serve_monthly_df = pd.read_csv('Sample Data/cost_to_serve_monthly.csv')

In [25]:
episodes_df.to_sql('episodes', conn, if_exists='replace', index=False)
organizations_monthly_df.to_sql('organizations_monthly', conn, if_exists='replace', index=False)
cost_to_serve_monthly_df.to_sql('cost_to_serve_monthly', conn, if_exists='replace', index=False)

12

## Utilization Analysis

Previewing Tables

In [26]:
# episodes table
episodes_preview = pd.read_sql_query("SELECT * FROM episodes LIMIT 5;", conn)
print("Episodes Preview:")
display(episodes_preview)

# organizations_monthly table
org_monthly_preview = pd.read_sql_query("SELECT * FROM organizations_monthly LIMIT 5;", conn)
print("\nOrganizations Monthly Preview:")
display(org_monthly_preview)

# cost_to_serve_monthly table
cost_serve_monthly_preview = pd.read_sql_query("SELECT * FROM cost_to_serve_monthly LIMIT 5;", conn)
print("\nCost to Serve Monthly Preview:")
display(cost_serve_monthly_preview)

Episodes Preview:


,EPISODE_ID,ORGANIZATION_ID,EPISODE_CREATED_AT,PROGRAM,OUTCOME
0,d0d178b23004534a5ff6723959785ba20ce1971d,5,2023-04-10T12:31:41.107Z,primary_care,walkin_clinic
1,ea85cffb681a06788b364d9ce49b61649f8688a5,5,2023-05-26T12:19:44.878Z,primary_care,walkin_clinic
2,7624c12a4ef8847e1951c0918d7193a9c3326dc1,5,2023-06-02T11:22:07.744Z,primary_care,md_np_appointment
3,9d994173c5775829c6b6badd898972522f038794,5,2023-11-24T14:00:17.658Z,primary_care,mhs_coaching
4,6be180209d276715e9256a78e8a73b13fe6a460d,2,2023-02-25T12:20:49.917Z,primary_care,md_np_appointment



Organizations Monthly Preview:


,DATE_MONTH,ORGANIZATION_ID,ELIGIBLE_MEMBERS,PEPM,HAS_PRIMARY_CARE_SINCE,HAS_MENTAL_HEALTH_SINCE,HAS_EAP_SINCE
0,2023-10-01T00:00:00Z,6,185,7.47,2000-01-01T00:00:00Z,None,None
1,2023-06-01T00:00:00Z,6,219,7.47,2000-01-01T00:00:00Z,None,None
2,2023-01-01T00:00:00Z,1,268,15.71,2000-01-01T00:00:00Z,2021-01-19T22:58:24.2361Z,2021-11-01T16:05:28.0127Z
3,2023-03-01T00:00:00Z,1,272,15.71,2000-01-01T00:00:00Z,2021-01-19T22:58:24.2361Z,2021-11-01T16:05:28.0127Z
4,2023-09-01T00:00:00Z,4,1436,14.21,2000-01-01T00:00:00Z,2021-06-01T12:15:33.8532Z,2023-04-05T21:32:35.77486Z



Cost to Serve Monthly Preview:


,DATE_MONTH,COST_TO_SERVE_PRIMARY_CARE,COST_TO_SERVE_MENTAL_HEALTH,COST_TO_SERVE_EAP
0,2023-01-01,60.04,334.29,112.27
1,2023-02-01,62.90,312.29,111.19
2,2023-03-01,62.96,298.27,119.29
3,2023-04-01,63.77,320.80,120.93
4,2023-05-01,65.27,276.54,114.95


Number of episodes by category for the year 2023

In [27]:
query = """
SELECT PROGRAM, COUNT(*) AS EPISODES_COUNT
FROM episodes
WHERE ORGANIZATION_ID = 4
GROUP BY PROGRAM;
"""
df = pd.read_sql_query(query, conn)
display(df)

,PROGRAM,EPISODES_COUNT
0,eap,67
1,mental_health,112
2,primary_care,550


Episodes opened by month

In [28]:
query_monthly_episodes = """
SELECT
  strftime('%Y-%m', EPISODE_CREATED_AT) AS month,
  COUNT(EPISODE_ID) AS episodes_opened
FROM episodes
WHERE organization_id = 4
GROUP BY month
ORDER BY month;


"""

df = pd.read_sql_query(query_monthly_episodes, conn)
display(df)

,month,episodes_opened
0,2023-01,1
1,2023-02,1
2,2023-05,158
3,2023-06,89
4,2023-07,78
5,2023-08,76
6,2023-09,58
7,2023-10,90
8,2023-11,90
9,2023-12,88


Eligible members per month

In [29]:
query_monthly_members = """
SELECT
  strftime('%Y-%m', date_month) AS month,
  eligible_members
FROM organizations_monthly
WHERE organization_id = 4
GROUP BY month
ORDER BY month;
"""

df = pd.read_sql_query(query_monthly_members, conn)
display(df)

,month,ELIGIBLE_MEMBERS
0,2023-01,59
1,2023-02,43
2,2023-03,42
3,2023-04,43
4,2023-05,1478
5,2023-06,1466
6,2023-07,1453
7,2023-08,1429
8,2023-09,1436
9,2023-10,1497
